# Protein Classification for Kunitz domain with HMM
Pfam family entry: [Kunitz_BPTI](https://pfam.xfam.org/family/Kunitz_BPTI) (PF00014)

This doc lives in:

In [1]:
%%bash
pwd

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/docs


## Notes for this prj from LB1/prj_blast_classification/

### blastclust
blastclust -i ids_kunitz.fasta -o ids_kunits.clust -L 0.8 -S 90

clusters together the sequences that share 90% sequence identity for 0.8 sequence length(?). The output is sorted such that in the first line there is the cluster with the greatest number of sequences

This will be very useful for checking and removing redundancy when considering queries in PDB

### HMM classificator framework
* fetch pdb for kunitz and non-kunitz
* check for redundancy and filter
* train hmm with training set
* optimize threshold
* test hmm with different set (do some ten-fold cross-validation)
* do further tests that will be discussed later

### Fetch pdb files
* Use RCSB PDB advanced search; note that you can filter for mutations, co-crystallization...
* Use pdbefold to have a list of structures given a selected one

### Report sections
* Datasets
* Model Building
* Optimization
* Performance

### Misc
* Consider Python "set" function to perform operations on sets; intersection, union etc. e.g. useful to compare lists of identifiers
* This might be useful for finding deprecated software packages: http://lipid.biocomp.unibo.it/emidio/tmp/

## Manual, boring workflow

### pdb search

Filters:
* pfam domain 
* wildtype True
* resolution < 3.5 A
* sequence length, not to have big gaps in alignments: 50 - 90

**params**

Pfam Accession Number PF00014 and Resolution is between 0.0 and 3.5 and Sequence Length is between 50 and 90 and Percent Sequence Alignment Search : PDB can contain Expression Tag sequence = Yes , 

From the advanced search --> get results of the query --> select custom tabular report with customizable info
5PTI seems to be a good candidate for being a representative of the struture
* no other proteins linked --> active conformation
* good resolution
* has ligands, should not be a problem

Prof pbtained note: 163 unique id_chain

### pdbefold search
Submit 5PTI search

Note: 569 unique id_chain

### match the search from pdb with the search from pdbefold
Take the intersection

Side note: to implement queue management of jobs [Sun Grid Engine](https://en.wikipedia.org/wiki/Oracle_Grid_Engine)

## Handmade files
I downloaded the pdb search results and the pdbefold search results

In [2]:
%%bash
cd ../dataset/
pwd
ls -lh

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
total 0
lrwxrwxrwx 1 alessandro alessandro 35 apr  3 11:40 pdbefold_5pti_A_summary -> ../handmade/pdbefold_5pti_A_summary
lrwxrwxrwx 1 alessandro alessandro 43 apr  3 12:16 rcsbpdb_wt_PF00014_len50_90.csv -> ../handmade/rcsbpdb_wt_PF00014_len50_90.csv
lrwxrwxrwx 1 alessandro alessandro 23 apr  3 09:25 Snakefile -> ../snakefiles/Snakefile


### Parsing handmade files
#### rcsbpdb_wt_PF00014_len50_90.txt.gz
.META:

1 ID_CHAIN

2 Resolution

3 Taxonomy

4 Sequence

5 Chain Length

#### pdbefold_5pti_A_summary.txt.gz
.META:
1 ID_CHAIN

2 Q-score

3 P-score

4 Z-score

5 RMSD

In [3]:
%%bash
cd ../dataset/
pwd
snakemake -p rcsbpdb_wt_PF00014_len50_90.txt.gz
snakemake -p pdbefold_5pti_A_summary.txt.gz
zcat rcsbpdb_wt_PF00014_len50_90.txt.gz | head
zcat pdbefold_5pti_A_summary.txt.gz | head

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
1AAP_A	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58
1AAP_B	1.50	Homo_sapiens	VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA	58
1B0C_A	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_B	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_C	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_D	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1B0C_E	2.80	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1BHC_A	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1BHC_B	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
1BHC_C	2.70	Bos_taurus	RPDFCLEPPYTGPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA	58
5PTI_A	1	16.18	11.93	0.000
9PTI_A	0.999	14.33	11.2

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	parse_rcsbpdb_table
	1

rule parse_rcsbpdb_table:
    input: rcsbpdb_wt_PF00014_len50_90.csv
    output: rcsbpdb_wt_PF00014_len50_90.txt.gz
    jobid: 0

cat rcsbpdb_wt_PF00014_len50_90.csv | sed 's/,/	/g' | sed 's/"//g' | sed 's/ /_/g' | sed '1d' | sed 's/	/_/' | gzip > rcsbpdb_wt_PF00014_len50_90.txt.gz
Finished job 0.
1 of 1 steps (100%) done
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	parse_pdbefold_summary
	1

rule parse_pdbefold_summary:
    input: pdbefold_5pti_A_summary
    output: pdbefold_5pti_A_summary.txt.gz
    jobid: 0

cat pdbefold_5pti_A_summary | sed '1,5d' | tr -s " " "\t" | cut -f3- | awk '{print toupper($17)"	"$1"	"$2"	"$3"	"$4}' | sed 's/:/_/' | gzip > pdbefold_5pti_A_summary.txt.gz
Finished job 0.
1 of 1 steps (100%) done


## Download seqres from wwPDB
wget this: http://lipid.biocomp.unibo.it/emidio/tmp/pdb_seqres.txt

Not really needed since the sequence are already stored in rcsbpdb_wt_PF00014_len50_90.csv

## run blastclust on intersection set 
Prof obtained 25 clusters using -S 99 -L 0.95; parameters are flexible, but you want at least 20 clusters

Sort the identifiers from the clusters by resolution (or other custom parameter, e.g the length): extract the representative one for each cluster